In [1]:
import os

In [2]:
os.chdir("/Users/kei/Projects/spykesim/drafts/")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import line_profiler
import cython
%matplotlib inline
%load_ext line_profiler

In [4]:
# Row: Neuron #, Col: Bin #
mat1 = np.array([
    [1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0]])
mat2 = np.array([
    [1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1],
    [1, 1, 1, 1, 0],
    [1, 1, 1, 0, 0],
    [1, 0, 0, 0, 0]])

# とりあえずシンプルなケースから実装してみる

In [31]:
import numpy as np
def editsim(mat1, mat2):
    return _simpleeditsim(mat1, mat2)
def _simpleeditsim(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            dp_table[col1 + 1, col2 + 1] = np.max([
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match])
    return dp_table[-1, -1]

def _simpleeditsim_withbp(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    bp_table = np.ones_like(dp_table) * (-1)
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            choices = [
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match
            ]
            bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
            dp_table[col1 + 1, col2 + 1] = np.max(choices)
            
    return dp_table, bp_table

def _simpleeditsim_tracebp(bp, mat1, mat2):
    nrow = bp.shape[0]
    ncol = bp.shape[1]
    row = nrow - 1
    col = ncol - 1
    # The first column is inserted just to avoid initialization error that may occur on concatination.
    alignment1 = np.zeros((mat1.shape[0], 1))
    alignment2 = np.zeros((mat1.shape[0], 1))
    zerovec = np.zeros(mat1.shape[0]) # which is corresponding to the null character.
    while True:
        if bp[row, col] == -1:
            # Eather of the strings tracing terminated
            break
        elif bp[row, col] == 2:
            alignment1 = np.c_[mat1[:, row - 1] * mat2[:, col - 1], alignment1]
            alignment2 = np.c_[mat1[:, row - 1] * mat2[:, col - 1], alignment2]
            row -= 1
            col -= 1
        elif bp[row, col] == 1:
            alignment1 = np.c_[zerovec, alignment1]
            alignment2 = np.c_[mat2[:, col - 1], alignment2]
            col -= 1
        elif bp[row, col] == 0:
            alignment1 = np.c_[mat1[:, row - 1], alignment1]
            alignment2 = np.c_[zerovec, alignment2]
            row -= 1
    return alignment1[:, :-1], alignment2[:, :-1]

def profile_simpleeditsim():
    nrow = 100
    mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    for i in range(10):
        a = _simpleeditsim(mat1, mat2)
        print(a)

In [33]:
%%timeit
profile_simpleeditsim

22.5 ns ± 0.166 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## 動作テスト

In [6]:
# A: 1-th col, T: 0-th col
# ATATATAT and
#  TATA AT
mat1 = np.array([
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0]
])

mat2 = np.array([
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0]
])

In [9]:
dp, bp = _simpleeditsim_withbp(mat1, mat2)
print(bp)
_simpleeditsim_tracebp(bp, mat1, mat2)

NameError: name '_simpleeditsim_withbp' is not defined

In [141]:
# ATATATAT and
#  TATA AT
mat1 = np.array([
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0]
])

mat2 = np.array([
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0]
])

# Profiling

In [10]:
nrow = 10
mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)

In [12]:
%%writefile profile_editsim_draft.py
import numpy as np
from editsim_draft import *
def profile():
    nrow = 10
    mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow)
    for i in range(10):
        a = editsim(mat1, mat2)

Overwriting profile_editsim_draft.py


In [28]:
import sys
import editsim_draft
from pathlib import Path
%lprun -T report.log -f editsim_draft._simpleeditsim editsim_draft.profile()
with Path("report.log").open( ) as rf: print(rf.readline())


*** Profile printout saved to text file 'report.log'. 
Timer unit: 1e-06 s



Timer unit: 1e-06 s

Total time: 0.036256 s
File: /Users/kei/Projects/spykesim/drafts/editsim_draft.py
Function: _simpleeditsim at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def _simpleeditsim(mat1, mat2):
     5        10         12.0      1.2      0.0      nrow = mat1.shape[1]
     6        10          7.0      0.7      0.0      ncol = mat2.shape[1]
     7        10         30.0      3.0      0.1      dp_table = np.zeros((nrow+1, ncol+1))
     8       110         60.0      0.5      0.2      for col1 in range(nrow):
     9      1100       1028.0      0.9      2.8          for col2 in range(ncol):
    10      1000       9366.0      9.4     25.8              match = np.dot(mat1[:, col1], mat2[:, col2])
    11      1000        943.0      0.9      2.6              dp_table[col1 + 1, col2 + 1] = np.max([
    12      1000        982.0      1.0      2.7                  dp_table[col1, col2 + 1],
    13      1000    

In [23]:
editsim_draft.profile()

In [ ]:
def ceval_edit_dist(fragment1, fragment2, dp_table, jitter, score_vec):
    return ceval_edit_dist_virtual(fragment1.astype(np.int32), fragment2.astype(np.int32), dp_table.astype(np.float32), jitter, 
                                 score_vec.astype(np.int32))
def ceval_edit_dist_virtual(np.ndarray[np.int32_t, ndim=2] mat1, np.ndarray[np.int32_t, ndim=2] mat2, np.ndarray[np.float32_t, ndim=2] dp_table, int jitter, 
                   np.ndarray[np.int32_t, ndim=1] score_vec):
    cdef int nrow = mat1.shape[0]
    cdef int ncol = mat1.shape[1]
    cdef int col1, col2
    cdef int row
    cdef float match=0
    cdef float disim
    for col1 in range(ncol):
        for col2 in range(int_max(0, col1-jitter), int_min(col1+jitter, ncol)):
            match = 0
            # calculate inner product
            for row in range(nrow):
                match += mat1[row, col1] * mat2[row, col2]
                match /= score_vec[row] + 1
            disim = nrow -  match
            # idx 0 : ←, 1 : ↑, 2, ↖
            dp_table[col1+1, col2+1] = float_min(dp_table[col1][col2+1]+nrow, dp_table[col1+1][col2]+nrow, dp_table[col1][col2]+disim)
    return dp_table[-1, -1]